# Using _osmium_ to count features in Nepal

In [2]:
import osmium as o
import numpy as np
import sys, os, json, pprint, copy
from collections import Counter

In [8]:
class FileHandler(o.SimpleHandler):                                                                                           
    def __init__(self):
        o.SimpleHandler.__init__(self)                                                                                              
        self.w_cnt = 0
        self.n_cnt = 0

#     def node(self, n):
#         '''
#             Only save nodes which have extra attributes...
#             TODO: If the first version of a node has 0 tags, but version 2 has tags, then version 1 may not show up in the history... maybe?
#         '''
#         self.n_cnt +=1
#         if n.id in nodes:
#             nodes[n.id].add(n)
#         elif len(n.tags)>0:
#             nodes[n.id] = Node(n)
            
#         #Cache these locations:
#         if n.id not in node_locations:
#             node_locations[n.id] = []

#         node_locations[n.id].append({
#                 'v':n.version,
#                 'c':n.changeset,
#                 'g':[n.location.lon, n.location.lat]
#             })
        
#         if self.n_cnt%10000==0:
#             sys.stderr.write("\r{0} nodes processed".format(self.n_cnt))

    def way(self, w):
        self.w_cnt += 1
    
        if w.id in ways: 
            ways[w.id] += 1
        else:
            ways[w.id] = 1
        if self.w_cnt%1000==0:
            sys.stderr.write("\r{0} ways processed".format(self.w_cnt))

Run the handler to create the objects

## Now process the objects

In [9]:
node_locations = {}
nodes = dict({})
ways  = dict({})
h = FileHandler()
h.apply_file('/data/osm/nepal.osh.pbf', locations=False)

3185000 ways processed

In [12]:
c = Counter(list(ways.values()))

In [13]:
c.most_common()

[(1, 2391544),
 (2, 184928),
 (3, 46689),
 (4, 17746),
 (6, 9889),
 (5, 8254),
 (7, 3275),
 (8, 1800),
 (9, 1176),
 (10, 850),
 (11, 609),
 (12, 471),
 (13, 350),
 (14, 264),
 (15, 243),
 (16, 178),
 (17, 155),
 (18, 120),
 (19, 91),
 (20, 79),
 (21, 65),
 (22, 59),
 (23, 45),
 (24, 42),
 (25, 35),
 (27, 30),
 (28, 28),
 (26, 26),
 (29, 18),
 (30, 16),
 (31, 15),
 (32, 14),
 (34, 13),
 (35, 12),
 (41, 12),
 (37, 9),
 (36, 8),
 (38, 8),
 (39, 8),
 (40, 7),
 (33, 6),
 (43, 4),
 (46, 4),
 (48, 3),
 (216, 3),
 (42, 2),
 (45, 2),
 (49, 2),
 (53, 2),
 (54, 2),
 (64, 2),
 (205, 2),
 (212, 2),
 (213, 2),
 (214, 2),
 (217, 2),
 (44, 1),
 (47, 1),
 (50, 1),
 (51, 1),
 (55, 1),
 (56, 1),
 (57, 1),
 (58, 1),
 (315, 1),
 (61, 1),
 (62, 1),
 (63, 1),
 (65, 1),
 (67, 1),
 (68, 1),
 (69, 1),
 (72, 1),
 (77, 1),
 (81, 1),
 (83, 1),
 (106, 1),
 (107, 1),
 (276, 1),
 (195, 1),
 (196, 1),
 (203, 1),
 (204, 1),
 (215, 1),
 (220, 1),
 (225, 1),
 (226, 1),
 (227, 1),
 (228, 1),
 (229, 1),
 (230, 1)]

In [ ]:
#Actually process the data:
for idx, (w_id, way) in enumerate(ways.items()):
    way.process_history()
    if idx%1000==0:
        sys.stderr.write("\r{0} ways processed".format(idx))

In [ ]:
for idx, (n_id, node) in enumerate(nodes.items()):
    node.process_history() 
    if idx%1000==0:
        sys.stderr.write("\r{0} nodes processed".format(idx))

In [ ]:
# Choose sample of data and print it
sample_ways  = np.random.choice(list(ways.keys()),10)
sample_nodes = np.random.choice(list(nodes.keys()),10)

In [ ]:
for way in [ways[w_id] for w_id in sample_ways]:
    print(way)
    pprint.pprint(way.as_geojson())
    
for node in [nodes[n_id] for n_id in sample_nodes]:
    print(node)
    pprint.pprint(node.as_geojson(geometries=False))
    pprint.pprint(node.as_geojson())

In [ ]:
# Write out the geojsonl file
with open('nepal_history_no_geometry.geojsonl','w') as outFile:
    for w_id, way in ways.items():
        outFile.write(json.dumps(way.as_geojson(geometries=False))+"\n")
    
    for n_id, node in nodes.items():
        outFile.write(json.dumps(node.as_geojson(geometries=False))+"\n")

Tippecanoe Instructions

    $ tippecanoe -o colorado_history_no_geometry.mbtiles -Pf -ps -pt -pf -pk -Z12 -z12 -d14 -l osm -n osm boulder_history.geojsonl

The geometries are stil _not quite right_... but they're close?